### Log Regression Model Tuning for 2016 - 2020 Data (imports)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import modelinghelper as helper

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import joblib
import os
from datetime import datetime

# supress future warnings
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [2]:
#set up random seed
rng = 42

# set up folder to save results
output_path = 'import_run8_TargetEncoding_gridsearchCV'
dataset = 'import'

if not os.path.exists(output_path):
    os.makedirs(output_path)
else:
    print("Folder already exists")

# file name
prefix = f'{output_path}/{dataset}'

Folder already exists


In [3]:
# read in data
p = '../data/'
input_name = 'cleaned.csv'

df = pd.read_csv(f'{p}{input_name}')

In [4]:
df['ship_date_mm'] = df['ship_date_mm'].astype(str)

In [5]:
df.describe()

,control_number,qty,value,qty_new,ship_date_yyyy
count,1.890132e+06,1.890132e+06,1.890132e+06,1.890132e+06,1.890132e+06
mean,2.018142e+09,1.955213e+06,9.658016e+03,1.954949e+06,2.017558e+03
std,1.263350e+06,2.644078e+08,9.355053e+05,2.644078e+08,1.279415e+00
min,2.014539e+09,-1.000000e+00,0.000000e+00,-1.000000e+00,2.016000e+03
25%,2.016978e+09,1.500000e+00,5.100000e+01,1.000000e+00,2.016000e+03
50%,2.017984e+09,9.000000e+00,3.180000e+02,8.000000e+00,2.017000e+03
75%,2.019505e+09,9.300000e+01,2.104000e+03,8.600000e+01,2.019000e+03
max,2.020755e+09,9.990708e+10,1.242236e+09,9.990708e+10,2.020000e+03


In [6]:
non_feature_cols = ['control_number', 'disp_date', 'i_e', 'ship_date', 
                    'cartons', 'qty', 'unit', 'specific_generic_name',
                    'genus', 'species', 'wildlf_cat', 
                    'disp_date_yyyy', 'disp_date_mm', 'disp_ship_date']

target = ['act', 'dp_cd']

feature_cols = ['species_code', 'wildlf_desc', 'wildlf_cat',
                'ctry_org', 'ctry_ie','purp', 'src', 'trans_mode', 'pt_cd', 
                 'value', 'ship_date_mm']

import_df = helper.df_filtering(df, i_e = 'I', f_cols = feature_cols)
# import: 1590505 rows × 11 columns
# export: 299340 rows × 11 columns
import_df.shape

(1590505, 12)

#### Modeling Pipeline

In [7]:
X_train, X_test, y_train, y_test = helper.data_split(import_df)

#### Log Regression

In [ ]:
clf = LogisticRegression(random_state=rng)
lr_params={
    'clf__class_weight':[None,
                         'balanced', 
                         {0:100, 1:1}, 
                         {0:50, 1:1}],
    'clf__solver': ['lbfgs', 'liblinear', 'sag', 'saga'],
    'clf__C': [0.1, 0.5, 1, 5, 10, 50, 100]
}

%time lr_pipe = helper.gridsearch_pipeline(X_train, y_train, clf, lr_params) 
lr_pipe

Fitting 5 folds for each of 112 candidates, totalling 560 fits
[CV 1/5] END clf__C=0.1, clf__class_weight=None, clf__solver=lbfgs;, score=0.454 total time=  10.3s
[CV 2/5] END clf__C=0.1, clf__class_weight=None, clf__solver=lbfgs;, score=0.463 total time=  10.4s
[CV 3/5] END clf__C=0.1, clf__class_weight=None, clf__solver=lbfgs;, score=0.442 total time=  10.6s
[CV 4/5] END clf__C=0.1, clf__class_weight=None, clf__solver=lbfgs;, score=0.446 total time=  11.0s
[CV 5/5] END clf__C=0.1, clf__class_weight=None, clf__solver=lbfgs;, score=0.441 total time=  10.3s
[CV 1/5] END clf__C=0.1, clf__class_weight=None, clf__solver=liblinear;, score=0.454 total time=  11.3s
[CV 2/5] END clf__C=0.1, clf__class_weight=None, clf__solver=liblinear;, score=0.463 total time=  11.6s
[CV 3/5] END clf__C=0.1, clf__class_weight=None, clf__solver=liblinear;, score=0.442 total time=  11.4s
[CV 4/5] END clf__C=0.1, clf__class_weight=None, clf__solver=liblinear;, score=0.446 total time=  11.7s
[CV 5/5] END clf__C=0

/opt/anaconda3/envs/test_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/5] END clf__C=0.1, clf__class_weight=None, clf__solver=sag;, score=0.450 total time=  47.3s


/opt/anaconda3/envs/test_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END clf__C=0.1, clf__class_weight=None, clf__solver=sag;, score=0.460 total time=  48.0s


...

[CV 5/5] END clf__C=50, clf__class_weight={0: 100, 1: 1}, clf__solver=sag;, score=0.095 total time=  52.6s


/opt/anaconda3/envs/test_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/5] END clf__C=50, clf__class_weight={0: 100, 1: 1}, clf__solver=saga;, score=0.097 total time=  53.1s


/opt/anaconda3/envs/test_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END clf__C=50, clf__class_weight={0: 100, 1: 1}, clf__solver=saga;, score=0.093 total time=  52.3s


In [ ]:
joblib.dump(lr_pipe, f'{prefix}_lr_pipe.joblib')

#### Evaluation

In [ ]:
outputs = True
lr_predicted = lr_pipe.predict(X_test)
confusion = confusion_matrix(y_test, lr_predicted, labels = [1,0])
if outputs:
    print('Log Regression (best model)\n', confusion)
cm_display = ConfusionMatrixDisplay(confusion, display_labels=[1,0])
if outputs:
    cm_display.plot()
    
    